In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, split

sc = pyspark.SparkContext('local[*]')


In [2]:
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('users_from_arizona.csv')
newDf = df.select(col("user_id"), split(col("friends"), ",\s*").alias("friends"))


def addToSet(x):
    user_friends_set = []
    for each in x.friends:
        if(x.user_id < each):
            user_friends_set.append((x.user_id,each))
        else:
            user_friends_set.append((each,x.user_id))
    return user_friends_set

rdd = newDf.rdd.flatMap(addToSet)
rdd.distinct().count()

13969027

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
final_df = rdd.distinct().toDF()
final_df.write.format('com.databricks.spark.csv').save('user_friends_flatmap_2lkh_record.csv')

In [6]:
print "Without using distinct"
rdd.count()

Without using distinct


14831855